In [135]:
# Some more magic so that the notebook will reload external python modules;
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
# This will automatically reload src/dtree.py every time you make changes and save the file
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [136]:
%matplotlib inline

In [137]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [138]:
# combine
# fill in make
# fill in mileage, age
# fill in power and engine cap
# fill in policy-related values except depreciation rate
# calculate price based on depreciation rate
# save the values with price and evaluate the performance

In [174]:
df1 = pd.read_csv('./train.csv')
df2 = pd.read_csv('./test.csv')
l = df2.columns.tolist()
df1 = df1[l]
df = pd.concat([df1, df2])
trainLen = len(df1)
def divideDf(df):
    train = df[:trainLen]
    test = df[trainLen:]
    return len(train), train, test

def combineDf(train, test):
    return pd.concat([train, test])

In [175]:
df1.info()
df2.info()
# ['depreciation', 'omv', 'arf', 'age', 'remaining_coe', 'dereg_value', 'truck', 'suv', 'sports car', 'luxury sedan',\
#          'bus/mini bus', 'mid-sized sedan', 'stationwagon', 'hatchback', 'mpv']

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16784 entries, 0 to 16783
Data columns (total 29 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   listing_id         16784 non-null  int64  
 1   title              16784 non-null  object 
 2   make               14624 non-null  object 
 3   model              16784 non-null  object 
 4   description        16439 non-null  object 
 5   manufactured       16590 non-null  float64
 6   original_reg_date  318 non-null    object 
 7   reg_date           16583 non-null  object 
 8   type_of_vehicle    16784 non-null  object 
 9   category           16784 non-null  object 
 10  transmission       16784 non-null  object 
 11  curb_weight        16205 non-null  float64
 12  power              14447 non-null  float64
 13  fuel_type          3490 non-null   object 
 14  engine_cap         16731 non-null  float64
 15  no_of_owners       16608 non-null  float64
 16  depreciation       163

In [176]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21784 entries, 0 to 4999
Data columns (total 29 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   listing_id         21784 non-null  int64  
 1   title              21784 non-null  object 
 2   make               18997 non-null  object 
 3   model              21784 non-null  object 
 4   description        21328 non-null  object 
 5   manufactured       21529 non-null  float64
 6   original_reg_date  418 non-null    object 
 7   reg_date           21522 non-null  object 
 8   type_of_vehicle    21784 non-null  object 
 9   category           21784 non-null  object 
 10  transmission       21784 non-null  object 
 11  curb_weight        21049 non-null  float64
 12  power              18772 non-null  float64
 13  fuel_type          4541 non-null   object 
 14  engine_cap         21716 non-null  float64
 15  no_of_owners       21552 non-null  float64
 16  depreciation       2121

df_valid = df.dropna(subset = ['fuel_type'])

missingfuel_type = df.fuel_type.isnull()
dfm = df[missingfuel_type]
missingModelAge = []
for idx, row in dfm.iterrows():
    model = row['model']
    missingModelAge.append(model)

mapper = {}
for idx, row in df_valid.iterrows():
    model = row['model']
    fuel_type = row['fuel_type']
    mapper[model] = fuel_type

for idx, row in dfm.iterrows():
    model = row['model']
    if model in mapper:
        df.at[idx, 'fuel_type'] = mapper[model]

print(df['fuel_type'].isnull().sum())

In [177]:
#fill in make
dfvalid = df.dropna(subset = ['make'])

missingMake = df.make.isnull()
dfm = df[missingMake]
missingModelAge = []
for idx, row in dfm.iterrows():
    model = row['model']
    missingModelAge.append(model)

mapper = {}
for idx, row in dfvalid.iterrows():
    model = row['model']
    make = row['make']
    mapper[model] = make

for idx, row in dfm.iterrows():
    model = row['model']
    df.at[idx, 'make'] = mapper[model]

print(df['make'].isnull().sum())

0


In [178]:
# fill in mileage
df_valid = df.dropna(subset = ['mileage', 'no_of_owners'])
no_of_ownerss = df_valid['no_of_owners'].to_numpy()
caps = df_valid['mileage'].to_numpy()

k = 1000
for idx, row in df.iterrows():
    if idx % 10000 == 0:
        print(idx)

    p, e = row['no_of_owners'], row['mileage']
    
    if type(row["mileage"]) == float and pd.isna(row["mileage"]):

        diff = np.abs(no_of_ownerss - p)

        indices = np.argsort(diff)[:k]

        k_caps = caps[indices]

        avg = np.average(k_caps)

        # Set the weight value of the current row to the median
        df.at[idx, 'mileage'] = avg
print(df['mileage'].isnull().sum())

0
10000
0
0


In [179]:
# get age and remaining coe
d1 = df['manufactured'].isnull()
df['age'] = df['manufactured'].copy()
df.loc[d1, 'age'] = df.loc[d1, 'manufactured']
d1 = df['age'].isnull()
df.loc[d1, 'age'] = df.loc[d1, 'original_reg_date']
d1 = df['age'].isnull()
df.loc[d1, 'age'] = df.loc[d1, 'reg_date']

def month(x):
    if pd.isna(x):
        return np.nan
    mapper = {}
    all_m = ['jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec']
    for i in range(12):
        mapper[all_m[i]] = i + 1
    if len(str(x)) < 11:
        return (2021 - int(x)) * 12
    s = x.split('-')
    year = 2021 - int(s[-1])
    month = mapper[s[-2]]
    return year * 12 + month

def filterNeg(x):
    if x < 0:
        return 0
    return x
    
df['age'] = df['age'].map(month)

d1 = df['reg_date'].isnull()
df['remaining_coe'] = df['reg_date'].copy()
df.loc[d1, 'remaining_coe'] = df.loc[d1, 'reg_date']
d1 = df['remaining_coe'].isnull()
df.loc[d1, 'remaining_coe'] = df.loc[d1, 'original_reg_date']
d1 = df['remaining_coe'].isnull()
df.loc[d1, 'remaining_coe'] = df.loc[d1, 'manufactured']
df['remaining_coe'] = 120 - df['remaining_coe'].map(month)
df['remaining_coe'] = df['remaining_coe'].map(filterNeg)

hasLifeSpan = ~df['lifespan'].isnull()
def get_remaining(x):
    diffFrom2021 = month(x)
    res = -diffFrom2021
    if res < 0:
        res = 0
    return res ##check 2021
df.loc[hasLifeSpan, 'remaining_coe'] = df.loc[hasLifeSpan, 'lifespan'].map(get_remaining)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21784 entries, 0 to 4999
Data columns (total 31 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   listing_id         21784 non-null  int64  
 1   title              21784 non-null  object 
 2   make               21784 non-null  object 
 3   model              21784 non-null  object 
 4   description        21328 non-null  object 
 5   manufactured       21529 non-null  float64
 6   original_reg_date  418 non-null    object 
 7   reg_date           21522 non-null  object 
 8   type_of_vehicle    21784 non-null  object 
 9   category           21784 non-null  object 
 10  transmission       21784 non-null  object 
 11  curb_weight        21049 non-null  float64
 12  power              18772 non-null  float64
 13  fuel_type          4541 non-null   object 
 14  engine_cap         21716 non-null  float64
 15  no_of_owners       21552 non-null  float64
 16  depreciation       2121

In [180]:
# get omv
from collections import defaultdict
df_valid = df.dropna(subset = ['omv'])
# df_valid = pd.concat([df_valid, df_train], axis = 0)
# print(df_valid.shape)
omvs = df_valid['omv'].to_numpy()


# missingOmvModelsAge (1 year)
# missingomv = avg(modelAgeToSimilarOmv)
missingOmv = df.omv.isnull()
dfm = df[missingOmv]
missingModelAge = []
for idx, row in dfm.iterrows():
    model = row['model']
    age = row['age']
    p = (model, age)
    missingModelAge.append(p)

mapper = defaultdict(list)
for idx, row in df_valid.iterrows():
    model = row['model']
    age = row['age']
    omv = row['omv']
    p = (model, age)
    mapper[p].append(omv)

for idx, row in dfm.iterrows():
    model = row['model']
    age = row['age']
    p = (model, age)
    if len(mapper[p]):
        df.at[idx, 'omv'] = np.average(mapper[p])

In [181]:
print(len(df[df.omv.isnull()]))
# print(df[missingOmv]['category'].unique())
# print(df[missingOmv][df.category.str.contains('low mileage')])
# print(df[missingOmv][df.age < 120]['category'])
# print(df[missingOmv])

45


In [182]:
df_valid = df.dropna(subset = ['omv'])
# df_valid = pd.concat([df_valid, df_train], axis = 0)
# print(df_valid.shape)
omvs = df_valid['omv'].to_numpy()


# missingOmvModelsAge (1 year)
# missingomv = avg(modelAgeToSimilarOmv)
missingOmv = df.omv.isnull()
dfm = df[missingOmv]
missingModelAge = []
for idx, row in dfm.iterrows():
    model = row['model']
    age = row['age'] // 24 * 24
    p = (model, age)
    missingModelAge.append(p)

mapper = defaultdict(list)
for idx, row in df_valid.iterrows():
    model = row['model']
    age = row['age'] // 24 * 24
    omv = row['omv']
    p = (model, age)
    mapper[p].append(omv)

for idx, row in dfm.iterrows():
    model = row['model']
    age = row['age'] // 24 * 24
    p = (model, age)
    if len(mapper[p]):
        df.at[idx, 'omv'] = np.average(mapper[p])

In [183]:
df = df_.copy()
print(len(df))
print(len(df) - 5000)
train = df[:trainLen].dropna(subset=['omv'])
trainLen = len(train)
print(trainLen)
test = df[16784:].copy()
df = pd.concat([train, test])
print(len(test))

21784
16784
16751
5000


In [184]:
test_df = df[-5000:]
omvL = test_df[test_df.omv.isnull()].index.tolist()
print(omvL)
test_df['omv'] = test_df['omv'].fillna(0)

df = pd.concat([df[:-5000], test_df])
# prices = [120000, 33800, 109888, 718000, 718000, 109888] ####
# df[:trainLen] = df[:trainLen].dropna(subset = ['omv'])

[706, 1026, 1770, 1889, 1904, 2015, 2062, 2282, 2633, 3224]


G:\softwares\envs\4277lab\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [185]:
# age,power, enginecap
df.loc[df.age < 0, 'age'] = 0
df.loc[df.age > 20000, 'age'] = 0

In [186]:
##train
print(df[df.engine_cap == 0])
df.at[8199, 'engine_cap'] = np.nan

      listing_id                                              title  \
8199     1005670  Mercedes-Benz E-Class E300e Plug-in Hybrid AMG...   

               make model                                        description  \
8199  mercedes-benz   amg  great deal! grab it! new facelifted 2021 with ...   

      manufactured original_reg_date     reg_date type_of_vehicle  \
8199        2021.0               NaN  16-jun-2021    luxury sedan   

                                               category  ...      omv  \
8199  parf car, almost new car, low mileage car, hyb...  ...  52492.0   

          arf  opc_scheme lifespan   eco_category  \
8199  66486.0         NaN      NaN  uncategorized   

                                               features  \
8199  2.0l m274 powerplant with eq boost producing 3...   

                                            accessories  indicative_price  \
8199  amg kits and wheels, memory seat, ambient ligh...               NaN   

      age  remaining_coe  
81

In [187]:
df_valid = df.dropna(subset = ['power'])
ps = df_valid['power'].to_numpy()

missingPs = df.power.isnull()
dfm = df[missingPs]
missingPower = []
for idx, row in dfm.iterrows():
    model = row['model']
    missingPower.append(model)

mapper = defaultdict(list)
for idx, row in df_valid.iterrows():
    model = row['model']
    power = row['power']
    mapper[model].append(power)

for idx, row in dfm.iterrows():
    model = row['model']
    if len(mapper[model]):
        df.at[idx, 'power'] = np.average(mapper[model])

print(df['power'].isnull().sum())

1982


In [188]:
df_valid = df.dropna(subset = ['engine_cap'])
ps = df_valid['engine_cap'].to_numpy()

missingPs = df.power.isnull()
dfm = df[missingPs]
missingPower = []
for idx, row in dfm.iterrows():
    model = row['model']
    missingPower.append(model)

mapper = defaultdict(list)
for idx, row in df_valid.iterrows():
    model = row['model']
    power = row['engine_cap']
    mapper[model].append(power)

for idx, row in dfm.iterrows():
    model = row['model']
    if len(mapper[model]):
        df.at[idx, 'engine_cap'] = np.average(mapper[model])

print(df['engine_cap'].isnull().sum())

67


In [189]:
df_valid = df.dropna(subset = ['power', 'engine_cap'])
powers = df_valid['power'].to_numpy()
caps = df_valid['engine_cap'].to_numpy()

k = 100

# Loop over each data point in our dataset
for idx, row in df.iterrows():
    p, e = row['power'], row['engine_cap']
    
    if type(row["power"]) == float and pd.isna(row["power"]):
        diff = np.abs(caps - e)

        indices = np.argsort(diff)[:k]

        k_powers = powers[indices]

        avg = np.average(k_powers)

        # Set the weight value of the current row to the median
        df.at[idx, 'power'] = avg

In [190]:
df_valid = df.dropna(subset = ['power', 'engine_cap'])
powers = df_valid['power'].to_numpy()
caps = df_valid['engine_cap'].to_numpy()

k = 50
for idx, row in df.iterrows():
    if idx % 10000 == 0:
        print(idx)

    p, e = row['power'], row['engine_cap']
    
    if type(row["engine_cap"]) == float and pd.isna(row["engine_cap"]):

        diff = np.abs(powers - p)

        indices = np.argsort(diff)[:k]

        k_caps = caps[indices]

        avg = np.average(k_caps)

        # Set the weight value of the current row to the median
        df.at[idx, 'engine_cap'] = avg

0
10000
0


df_ = df.copy()
df_['efficiency'] = df_['power']
for i in range(len(df_)):
    x = df_.iloc[i]
    df_['efficiency'][i] = x['power'] / x['engine_cap']

plt.boxplot(df_[df_.fuel_type == 'diesel']['efficiency'])# / diesel['engine_cap'])
plt.show()

In [23]:
# plt.boxplot(df['power'])# / diesel['engine_cap'])
# plt.show()

In [24]:
# plt.boxplot(df['engine_cap'])# / diesel['engine_cap'])
# plt.show()

plt.boxplot(df_[df_.fuel_type == 'petrol']['efficiency'])# / diesel['engine_cap']) ##outlier
plt.show()

# fill omv, get data with missing omv and dereg_value, fill in the omv
is_ = df[df.omv.isnull() & df.dereg_value.isnull()].index.tolist()
filled_omvs = []
for idx in is_:
    ag = df.at[idx, 'age']
    price = df.at[idx, 'price']
    mo = reg.predict(np.array([[ag, price]]))
    filled_omvs.append(mo)
for i, idx in enumerate(is_):
    df.at[idx, 'omv'] = filled_omvs[i]

In [ ]:
print(df['coe'].isnull().sum())

In [ ]:
plt.boxplot(df['coe'].dropna())
plt.show()

In [191]:
# coe: fill in missing value for coe according to: age and power
fill_arf = df[df.arf.isnull()]
for idx, row in fill_arf.iterrows():
    omv = row['omv']
    if omv < 20000:
        df.at[idx, 'arf'] = omv
    elif omv >= 20000 and omv <= 50000:
        df.at[idx, 'arf'] = (omv - 20000) * 1.40 + 20000
    else:
        df.at[idx, 'arf'] =  (omv - 50000) * 1.80 + 62000
print(len(df[df.arf.isnull()]))
# df = df[~(df.arf.isnull() & df.dereg_value.isnull())] # remove values whose dereg_value cannot be calculated

0


In [192]:
# for date missing both coe and dereg_value, record it's month engine and power
coe_to_search = []
to_search = df[df.coe.isnull() & df.dereg_value.isnull()]

for idx, row in to_search.iterrows():
    # category -> A / B
    power = row["power"]
    engine_cap = row["engine_cap"]
    cat = 2
    month = row["age"]
    if engine_cap <= 1600 and power <= 97:
        cat = 1
    coe_to_search.append((cat, month))

coe_to_search = list(set(coe_to_search))
print(len(coe_to_search))

32


In [193]:
print(len(df[df.coe.isnull()]))

1134


In [194]:
t_to_coe = {}

for t in coe_to_search:
    t_to_coe[str(t)] = []

remains = df[~df.coe.isnull()]

for idx, row in remains.iterrows():
    if idx % 10000 == 0:
        print(idx)

    power = row["power"]
    engine_cap = row["engine_cap"]
    cat = 2
    month = row["age"]
    if engine_cap <= 1600 and power <= 97:
        cat = 1
    
    t = (cat, month)
    
    coe = row["coe"]
    if t in coe_to_search:
        t_to_coe[str(t)].append(coe)
        
c = 0
for idx, row in df.iterrows():
    power = row["power"]
    engine_cap = row["engine_cap"]
    cat = 2
    month = row["age"]
    if engine_cap <= 1600 and power <= 97:
        cat = 1
    t = str((cat, month))

    if type(row["coe"]) == float and pd.isna(row["coe"]):
        if t in t_to_coe:
            l = t_to_coe[t]
            if len(l) > 0:
                avg = sum(l) / len(l)
                df.at[idx, 'coe'] = avg
                c += 1
print(c)

0
10000
1130


In [195]:
print(len(df[df.coe.isnull()]))

4


In [196]:
avg = np.average(df[~df.coe.isnull()]['coe'])
df['coe'] = df['coe'].fillna(avg)
print(df['coe'].isnull().sum())

0


In [197]:
df.info()
df.to_csv('combined1.csv')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21751 entries, 0 to 4999
Data columns (total 31 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   listing_id         21751 non-null  int64  
 1   title              21751 non-null  object 
 2   make               21751 non-null  object 
 3   model              21751 non-null  object 
 4   description        21295 non-null  object 
 5   manufactured       21497 non-null  float64
 6   original_reg_date  415 non-null    object 
 7   reg_date           21490 non-null  object 
 8   type_of_vehicle    21751 non-null  object 
 9   category           21751 non-null  object 
 10  transmission       21751 non-null  object 
 11  curb_weight        21040 non-null  float64
 12  power              21751 non-null  float64
 13  fuel_type          4539 non-null   object 
 14  engine_cap         21751 non-null  float64
 15  no_of_owners       21520 non-null  float64
 16  depreciation       2118

In [306]:
df = pd.read_csv('combined1.csv')

In [307]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21751 entries, 0 to 21750
Data columns (total 32 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         21751 non-null  int64  
 1   listing_id         21751 non-null  int64  
 2   title              21751 non-null  object 
 3   make               21751 non-null  object 
 4   model              21751 non-null  object 
 5   description        21295 non-null  object 
 6   manufactured       21497 non-null  float64
 7   original_reg_date  415 non-null    object 
 8   reg_date           21490 non-null  object 
 9   type_of_vehicle    21751 non-null  object 
 10  category           21751 non-null  object 
 11  transmission       21751 non-null  object 
 12  curb_weight        21040 non-null  float64
 13  power              21751 non-null  float64
 14  fuel_type          4539 non-null   object 
 15  engine_cap         21751 non-null  float64
 16  no_of_owners       215

In [308]:
# train = df[:-5000].copy()
# prices = pd.read_csv('train.csv')['price']
# prices = prices[train.index]

# for idx in range(len(train)):
#     row = train.iloc[idx]
#     depreciation = row['depreciation']
#     age = row["age"]
#     coe = row["coe"]
#     omv = row["omv"]
#     arf = row["arf"]
#     remaining_coe = row['remaining_coe']

#     min_parf = min(0.55 * omv, arf)
#     if age <= 96:
#         min_parf = 0.5 * arf ####
#     elif age > 96 and age <= (2021 - 2008) * 12:
#         min_parf = min(omv, arf)
    
#     # print(idx, len(prices))
#     new_coe = (prices.iloc[idx] - 0.5 * min_parf) / depreciation
#     if new_coe <= 120 and new_coe >= 0 and age >= (2021 - 2002) * 12:
#         train.iloc[idx, 31] = new_coe


In [309]:
# df = pd.concat([train, df[-5000:]])
# df.info()
df = df[-5000:]
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 16751 to 21750
Data columns (total 32 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         5000 non-null   int64  
 1   listing_id         5000 non-null   int64  
 2   title              5000 non-null   object 
 3   make               5000 non-null   object 
 4   model              5000 non-null   object 
 5   description        4889 non-null   object 
 6   manufactured       4939 non-null   float64
 7   original_reg_date  100 non-null    object 
 8   reg_date           4939 non-null   object 
 9   type_of_vehicle    5000 non-null   object 
 10  category           5000 non-null   object 
 11  transmission       5000 non-null   object 
 12  curb_weight        4844 non-null   float64
 13  power              5000 non-null   float64
 14  fuel_type          1051 non-null   object 
 15  engine_cap         5000 non-null   float64
 16  no_of_owners       

In [301]:
# # diff = []
# for idx in range(len(df)):
#     if pd.isnull(row["dereg_value"]):
#         row = df.iloc[idx]
#         max_age = 10 * 12
#         age = row["age"]
#         time_left = max_age - age
#         coe = row["coe"]
#         omv = row["omv"]
#         arf = row["arf"]
#         parf_ratio = 0
#         if age <  5 * 12:
#             parf_ratio = 0.75
#         elif age >= 5 * 12 and age < 10 * 12:
#             parf_ratio = 0.75 - math.ceil((age - 5 * 12) / 12) * 0.05 
#         remaining_coe = row['remaining_coe']

#         coe_rebate = coe * remaining_coe / 120
#         dereg_value = parf_ratio * arf + coe_rebate
#         if dereg_value <= 0:
#             dereg_value = np.nan

#         df.iat[idx, 20] = dereg_value #####
# #     else:
# #         p = abs(row["dereg_value"] - dereg_value)
# #         print(row["dereg_value"], dereg_value)
# #         if idx == 20:
# #             break
# #         diff.append(p)

# print(df.dereg_value.isnull().sum())

335


In [310]:
deregL = df[df.dereg_value.isnull()].index.tolist()
print(len(deregL))
df['dereg_value'] = df['dereg_value'].fillna(0)
depL = df[df.depreciation.isnull()].index.tolist()
df['depreciation'] = df['depreciation'].fillna(0)

335


In [311]:
# ## train
# df_train = df[:-5000]
# df_test = df[-5000:]
# df_train.to_csv('train_raw.csv')
# df_test.to_csv('test_raw.csv')
df_test = df.copy()

In [312]:
useful = ['mileage', 'depreciation', 'omv', 'age', 'remaining_coe', 'dereg_value', 'truck', 'suv', 'sports car', 'luxury sedan',\
         'bus/mini bus', 'mid-sized sedan', 'stationwagon', 'hatchback', 'mpv']
cars_dummies = pd.get_dummies(df_test['type_of_vehicle'])
df_test = pd.concat([df_test, cars_dummies], axis=1)
df_test = df_test[useful]
df_test.to_csv('./test/null_dep_dereg.csv')
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 16751 to 21750
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   mileage          5000 non-null   float64
 1   depreciation     4868 non-null   float64
 2   omv              5000 non-null   float64
 3   age              5000 non-null   int64  
 4   remaining_coe    5000 non-null   int64  
 5   dereg_value      5000 non-null   float64
 6   truck            5000 non-null   uint8  
 7   suv              5000 non-null   uint8  
 8   sports car       5000 non-null   uint8  
 9   luxury sedan     5000 non-null   uint8  
 10  bus/mini bus     5000 non-null   uint8  
 11  mid-sized sedan  5000 non-null   uint8  
 12  stationwagon     5000 non-null   uint8  
 13  hatchback        5000 non-null   uint8  
 14  mpv              5000 non-null   uint8  
dtypes: float64(4), int64(2), uint8(9)
memory usage: 278.4 KB


In [213]:
# l = df_test[~df_test.depreciation.isnull()]['listing_id'].tolist()
# print(l)
df_test = pd.read_csv('./test/with_dep.csv')

df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Unnamed: 0       5000 non-null   int64  
 1   depreciation     5000 non-null   float64
 2   omv              5000 non-null   float64
 3   arf              5000 non-null   float64
 4   age              5000 non-null   int64  
 5   remaining_coe    5000 non-null   int64  
 6   dereg_value      5000 non-null   float64
 7   truck            5000 non-null   int64  
 8   suv              5000 non-null   int64  
 9   sports car       5000 non-null   int64  
 10  luxury sedan     5000 non-null   int64  
 11  bus/mini bus     5000 non-null   int64  
 12  mid-sized sedan  5000 non-null   int64  
 13  stationwagon     5000 non-null   int64  
 14  hatchback        5000 non-null   int64  
 15  mpv              5000 non-null   int64  
dtypes: float64(4), int64(12)
memory usage: 625.1 KB


In [ ]:

# # depreciation: model, age, mileage based on df
# mam_dep = defaultdict(list)
# for idx, row in df.iterrows():
#     #500
#     m1 = row['model']
#     m2 = row['mileage'] // 500 * 500
#     a = row['age'] // 18 * 18
#     t = (m1,m2,a)
#     mam_dep[t].append(row['depreciation'])

# for idx, row in df_test.iterrows():
#     #500
#     dep = row['depreciation']
#     if type(dep) == float and pd.isna(dep):
#         m1 = row['model']
#         m2 = row['mileage'] // 500 * 500
#         a = row['age'] // 18 * 18
#         t = (m1,m2,a)
#         df_test.at[idx, 'depreciation'] = np.average(mam_dep[t])
# print(df_test['depreciation'].isnull().sum())

In [ ]:
# # depreciation: model, age, mileage based on df
# mam_dep = defaultdict(list)
# for idx, row in df.iterrows():
#     #500
#     m1 = row['model']
#     m2 = row['mileage'] // 1000 * 1000
#     a = row['age'] // 48 * 48
#     t = (m1,m2,a)
#     mam_dep[t].append(row['depreciation'])

# for idx, row in df_test.iterrows():
#     #500
#     dep = row['depreciation']
#     if type(dep) == float and pd.isna(dep):
#         m1 = row['model']
#         m2 = row['mileage']// 1000 * 1000
#         a = row['age'] // 48 * 48
#         t = (m1,m2,a)
#         df_test.at[idx, 'depreciation'] = np.average(mam_dep[t])
# print(df_test['depreciation'].isnull().sum())

In [ ]:
# # depreciation: model, age, mileage based on df
# mam_dep = defaultdict(list)
# for idx, row in df.iterrows():
#     #500
#     m1 = row['make']
#     m2 = row['mileage'] // 500 * 500
#     a = row['age'] // 18 * 18
#     p = row['power'] // 20 *20
#     t = (m1,m2,a, p)
#     mam_dep[t].append(row['depreciation'])

# for idx, row in df_test.iterrows():
#     #500
#     dep = row['depreciation']
#     if type(dep) == float and pd.isna(dep):
#         m1 = row['make']
#         m2 = row['mileage'] // 500 * 500
#         a = row['age'] // 18 * 18
#         p = row['power'] // 20 *20
#         t = (m1,m2,a, p)
#         df_test.at[idx, 'depreciation'] = np.average(mam_dep[t])
# print(df_test['depreciation'].isnull().sum())

In [ ]:
# # depreciation: model, age, mileage based on df
# mam_dep = defaultdict(list)
# for idx, row in df.iterrows():
#     #500
#     m1 = row['make']
#     m2 = row['mileage'] // 500 * 500
#     a = row['age'] // 18 * 18
#     p = row['power'] // 50 * 50
#     t = (m1,m2,a, p)
#     mam_dep[t].append(row['depreciation'])

# for idx, row in df_test.iterrows():
#     #500
#     dep = row['depreciation']
#     if type(dep) == float and pd.isna(dep):
#         m1 = row['make']
#         m2 = row['mileage'] // 500 * 500
#         a = row['age'] // 18 * 18
#         p = row['power'] // 50 * 50
#         t = (m1,m2,a, p)
#         df_test.at[idx, 'depreciation'] = np.average(mam_dep[t])
# print(df_test['depreciation'].isnull().sum())

In [ ]:
# # depreciation: model, age, mileage based on df
# mam_dep = defaultdict(list)
# for idx, row in df.iterrows():
#     #500
#     m1 = row['model']
#     a = row['age'] // 24 * 24
#     t = (m1,a)
#     mam_dep[t].append(row['depreciation'])

# for idx, row in df_test.iterrows():
#     #500
#     dep = row['depreciation']
#     if type(dep) == float and pd.isna(dep):
#         m1 = row['model']
#         a = row['age'] // 24 * 24
#         t = (m1,a)
#         df_test.at[idx, 'depreciation'] = np.average(mam_dep[t])
# print(df_test['depreciation'].isnull().sum())

In [ ]:
# # depreciation: model, age, mileage based on df
# mam_dep = defaultdict(list)
# for idx, row in df.iterrows():
#     #500
#     m1 = row['model']
#     a = row['age'] // 48 * 48
#     t = (m1,a)
#     mam_dep[t].append(row['depreciation'])

# for idx, row in df_test.iterrows():
#     #500
#     dep = row['depreciation']
#     if type(dep) == float and pd.isna(dep):
#         m1 = row['model']
#         a = row['age'] // 48 * 48
#         t = (m1,a)
#         df_test.at[idx, 'depreciation'] = np.average(mam_dep[t])
# print(df_test['depreciation'].isnull().sum())

In [ ]:
# # depreciation: model, age, mileage based on df
# mam_dep = defaultdict(list)
# for idx, row in df.iterrows():
#     #500
#     m1 = row['make']
#     a = row['age'] // 48 * 48
#     p = row['power'] // 50 * 50
#     t = (m1,a, p)
#     mam_dep[t].append(row['depreciation'])

# for idx, row in df_test.iterrows():
#     #500
#     dep = row['depreciation']
#     if type(dep) == float and pd.isna(dep):
#         m1 = row['make']
#         a = row['age'] // 48 * 48
#         p = row['power'] // 50 * 50
#         t = (m1,a, p)
#         df_test.at[idx, 'depreciation'] = np.average(mam_dep[t])
# print(df_test['depreciation'].isnull().sum())

In [248]:

# for idx, row in df_test.iterrows():
#     if type(row["dereg_value"]) == float and pd.isna(row["dereg_value"]):
#         max_age = 10 * 12
#         age = row["age"]
#         time_left = max_age - age
#         coe = row["coe"]
#         omv = row["omv"]
#         arf = row["arf"]
#         parf_ratio = 0
#         if age <  5 * 12:
#             parf_ratio = 0.75
#         elif age >= 5 * 12 and age < 10 * 12:
#             parf_ratio = 0.75 - math.ceil((age - 5 * 12) / 12) * 0.05 
#         remaining_coe = row['remaining_coe']
#         depreciation = row['depreciation']
#         coe_rebate = coe * remaining_coe / 120
#         dereg_value = parf_ratio * arf + coe_rebate
#         if dereg_value <= 0:
#             dereg_value = 0
#         df_test.at[idx, "dereg_value"] = dereg_value
# # df_test = df_test.dropna()
# df_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4863 entries, 0 to 4993
Data columns (total 17 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Unnamed: 0       4863 non-null   int64  
 1   Unnamed: 0.1     4863 non-null   int64  
 2   depreciation     4863 non-null   float64
 3   omv              4863 non-null   float64
 4   arf              4863 non-null   float64
 5   age              4863 non-null   int64  
 6   remaining_coe    4863 non-null   int64  
 7   dereg_value      4860 non-null   float64
 8   truck            4863 non-null   int64  
 9   suv              4863 non-null   int64  
 10  sports car       4863 non-null   int64  
 11  luxury sedan     4863 non-null   int64  
 12  bus/mini bus     4863 non-null   int64  
 13  mid-sized sedan  4863 non-null   int64  
 14  stationwagon     4863 non-null   int64  
 15  hatchback        4863 non-null   int64  
 16  mpv              4863 non-null   int64  
dtypes: float64(4),